In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from nltk import sent_tokenize

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
textsample = 'the matrix is everywhere its all around us, here even in this in room'

sentences = sent_tokenize(text=textsample)

In [ ]:
print(type(sentences))
print(sentences)

<class 'list'>
['the matrix is everywhere its all around us, here even in this in room']


In [2]:
import numpy as np
from scipy import sparse

dense = np.array([[[3,0,1],[0,2,0]]])
data = np.array([3,1,2])

row_pos = np.array([0,0,1])
col_pos = np.array([0,2,1])

sparse_coo = sparse.coo_matrix((data, (row_pos, col_pos)))

In [3]:
sparse_coo.toarray()

array([[3, 0, 1],
       [0, 2, 0]])

## 텍스트 분류 실습 예제

In [4]:
# Using fetch_20newsgropus api
# And make CSR and using Logistic Regression
# GridSearchCV and using scikit-learn Pipeline

from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset = 'all', random_state =111)

In [9]:
import pandas as pd

# 한 줄 처리
pd.Series(news_data.target)

0         6
1        13
2         6
3         9
4         5
         ..
18841     6
18842    12
18843     1
18844    12
18845    10
Length: 18846, dtype: int64

In [10]:
train_news = fetch_20newsgroups(subset = 'train', remove = ('headers', 'footers','quotes'), random_state = 111)

In [11]:
X_train  = train_news.data
y_train  = train_news.target

test_news = fetch_20newsgroups(subset='test',  remove = ('headers', 'footers','quotes'), random_state = 111)
X_test = test_news.data
y_test = test_news.target

len(test_news.data)

7532

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
# Import Model and Training model using default Parameter
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect =cnt_vect.transform(X_train)
X_test_cnt_vect= cnt_vect.transform(X_test)

In [15]:
print(f'학습 데이터 텍스트 shape: {X_train_cnt_vect.shape}')

학습 데이터 텍스트 shape: (11314, 101631)


In [16]:
# Evaluation

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

lr_classfier = LogisticRegression(solver = 'liblinear')
lr_classfier.fit(X_train_cnt_vect, y_train)
pred = lr_classfier.predict(X_test_cnt_vect)

ValueError: ignored

In [19]:
print(f'예측 정확도 {round(accuracy_score(y_test, pred),3)}')

예측 정확도 0.617


In [22]:
# Using TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect =tfidf_vect.transform(X_train)
X_test_tfidf_vect= tfidf_vect.transform(X_test)

# Using Logistic

lr_classfier = LogisticRegression(solver = 'liblinear')
lr_classfier.fit(X_train_tfidf_vect, y_train)
pred = lr_classfier.predict(X_test_tfidf_vect)

In [23]:
print(f'예측 정확도 {round(accuracy_score(y_test, pred),3)}')

예측 정확도 0.678


TF-IDF 방식의 Vector Feature 처리가 더 높은 예측 성능을 보여줬다.


일반적으로 텍스트가 많고 문서양이 많은 부분에 있어서는 TF-IDF 형식의 성능이 좋다.



In [25]:
 #Using Grid Search
from sklearn.model_selection import GridSearchCV

# dictionary class
params =  {'C':[0.01,0.1,1,5,10]}
# Model, params, cv, scoring part ..
grid_lr = GridSearchCV(lr_classfier, param_grid = params, cv=3, scoring='accuracy', verbose =1)
grid_lr.fit(X_train_tfidf_vect, y_train)
print(grid_lr.best_params_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
{'C': 10}


In [26]:
# Precict with best Condition
pred = grid_lr.predict(X_test_tfidf_vect)
print(f'예측 정확도: {accuracy_score(y_test, pred)}')

예측 정확도: 0.6894583112055231


In [29]:
# Data Pipeline for ML and this is example of pipeline

from sklearn.pipeline import Pipeline

pipeline = Pipeline([
  ('tfidf_vect',TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df =300)),
  ('lr_clf',LogisticRegression(solver='liblinear',C=10))
])

In [30]:
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)

In [31]:
print(f'Pipeline score: {round(accuracy_score(y_test, pred),2)}')

Pipeline score: 0.7


Logistic Regression 뿐만 아니라 SVM, Naive Bayes 등 다양한 분류 방식이 존재한다